<a href="https://colab.research.google.com/github/mtablado/uoc2022_tfm/blob/main/MRI-loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import os
import matplotlib
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# añade el path a tu directorio
%cd /content/drive/My Drive/tfm/dataset/IXI-T1

/content/drive/.shortcut-targets-by-id/1yKlrvb9Cp3amASiDUettfP3V1KJuzGDe/tfm/dataset/IXI-T1


In [4]:
import glob
import nibabel as nib #reading MR images

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def save_slices(filename :str, images :list):
  """
  Saves the slices given as an array on 3 different directories after being
  randomly shuffled to avoid overfitting.
  The 3 directoires at drive will be 'training', 'test', 'validation'
  Arguments:
    images: array with MRI brain slices
  """
  # Shuffle images to be stored randomly on training, validation and test dirs
  random.shuffle(images)

  total = len(images)
  print(total)
  for i, image in enumerate(images):
    name = filename + str(i) + '.png'
    if (i > 0.8 * total):
      name = 'images/test/' + name
    elif (i > 0.6 * total):
      name = 'images/validation/' + name
    else:
      name = 'images/training/' + name
    
    print("Saving file " + name)
    matplotlib.image.imsave(name, image, cmap='gray')

def load_slices(file :str) -> any:
  a = nib.load(file)
  a = a.get_fdata()
  a = a[:,80:100,:]
  return a

def extract_slices(file_list :list) -> np.ndarray:
  """ 
  It extracts the slices of the MRI images stored in the given files 
  Arguments:
      ff: list of nifti files
  Returns:
      An array with the selected slices from the different files  
  """
  
  for f in range(len(file_list)):
    filename = os.path.basename(file_list[f])
    images = []
    a = load_slices('./IXI-T1/' + filename)
    for i in range(a.shape[1]):
      images.append((a[:,i,:]))
    save_slices(filename, images)
  return images

try:
  os.mkdir('/content/drive/My Drive/tfm/dataset/images/training', mode=644)
  os.mkdir('/content/drive/My Drive/tfm/dataset/images/validation', mode=644)
  os.mkdir('/content/drive/My Drive/tfm/dataset/images/test', mode=644)
except:
  print('directories already exist')

# Make sure we are at the expected directory
%cd /content/drive/My Drive/tfm/dataset/

ff = glob.glob('./IXI-T1/*.nii.gz')

# Extract slices from files
images = extract_slices(ff)

image = images[1]

print (image.shape)

directories already exist
/content/drive/.shortcut-targets-by-id/1yKlrvb9Cp3amASiDUettfP3V1KJuzGDe/tfm/dataset
20
Saving file images/training/IXI023-Guys-0699-T1.nii.gz0.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz1.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz2.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz3.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz4.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz5.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz6.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz7.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz8.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz9.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz10.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz11.png
Saving file images/training/IXI023-Guys-0699-T1.nii.gz12.png
Saving file images/validation/IXI023-Guys-0699-T1.nii.gz13.png
Saving file images/validation/IXI023-Guy

In [ ]:
print(len(images))
print(images[1].shape)
print(type(image))

In [ ]:
print(image[100,50])

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
#img = image[:,:,100]
plt.imshow(image, cmap='gray')



In [ ]:
f, big_axes = plt.subplots(15,10, figsize=(25,25))

for i in range(0,15):
  for j in range(0,10):
    slice_position = ((i - 1) * 10) + j
    big_axes[i,j].imshow(image[:,:,slice_position], cmap='gray')


